In [9]:
!pip install datasets
from datasets import load_dataset, DatasetDict

from sklearn.model_selection import train_test_split

!pip install transformers
from transformers import AutoTokenizer, AutoConfig, pipeline, GPT2Tokenizer, GPT2Model, Trainer, TrainingArguments

!pip install gradio
import gradio as gr

!pip install accelerate -U
!pip install transformers[torch]
!pip install accelerator >=0.20.1

In [2]:
dataset = (load_dataset("gbharti/finance-alpaca", split='train').train_test_split(train_size=0.8, test_size=0.2))

dataset
# train_dataset, val_dataset = train_test_split(dataset["train"], test_size=0.2, random_state=42)
# print("Training examples:", len(train_dataset))
# print("Validation examples:", len(val_dataset))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/gbharti___json/gbharti--finance-alpaca-47c3412d84dc0065/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['instruction', 'text', 'input', 'output'],
        num_rows: 55129
    })
    test: Dataset({
        features: ['instruction', 'text', 'input', 'output'],
        num_rows: 13783
    })
})

In [3]:
model_checkpoint = "distilgpt2"
model = GPT2Model.from_pretrained(model_checkpoint)

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token

encoded_dataset = dataset.map(lambda examples: tokenizer(["instruction"], truncation=True, padding="max_length"))
encoded_dataset

Map:   0%|          | 0/55129 [00:00<?, ? examples/s]

Map:   0%|          | 0/13783 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'text', 'input', 'output', 'input_ids', 'attention_mask'],
        num_rows: 55129
    })
    test: Dataset({
        features: ['instruction', 'text', 'input', 'output', 'input_ids', 'attention_mask'],
        num_rows: 13783
    })
})

In [10]:
!pip show accelerate

Name: accelerate
Version: 0.20.3
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, packaging, psutil, pyyaml, torch
Required-by: 


In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

ImportError: ignored

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
)

NameError: ignored

In [ ]:
trainer.train()

In [ ]:
def predict(input_text):
    output = trainer.predict(input_text)
    return output


In [ ]:
interface = gr.Interface(fn=predict, inputs="text", outputs="text")
interface.launch(inline=False, debug=True)